In [43]:
#Third Year Paper Code
import numpy as np
import random 

In [18]:
#Basic Parameters
u = 100
β = 0.9
c_1 = 1
c_2 = 4
c_3 = 9
t = 1

In [ ]:
#initial value for Q
#When initial condition is production good in storage
#based on rational expectation
shock = 0
Q_12 = 65 + shock
Q_13 = 103 + shock
Q_11 = Q_12 + u
Q_21 = 256 + shock
Q_23 = 244 + shock
Q_22 = Q_23 + u
Q_31 = 290 + shock
Q_32 = 230 + shock
Q_33 = Q_31 + u

In [95]:
# Create agents
import random
# Agents have there type which same as there consumption type, they also have production type
class Agent:
    agentCount = 0

    def __init__(self, id, consume_type, product_type):
        self.id = id
        self.consume_type = consume_type
        self.product_type = product_type
        self.storage_type = product_type
        
        self.trading = False
        
        #generate type 1 agents' Q(precieved value for holding good 1 , 2 , 3)
        if self.consume_type == 1:
            self.Q_2 = 65 + random.randint(1, 10)
        if self.consume_type == 1:
            self.Q_3 = 103 + random.randint(1, 10)
        if self.consume_type == 1:
            self.Q_1 = 65 + u + random.randint(1, 10)
            
        #generate type 2 agents' Q(precieved value for holding good 1 , 2 , 3)
        if self.consume_type == 2:
            self.Q_1 = 256 + random.randint(1, 10)
        if self.consume_type == 2:
            self.Q_3 = 244 + random.randint(1, 10)
        if self.consume_type == 2:
            self.Q_2 = 244 + u + random.randint(1, 10)
            
        #generate type 3 agents' Q(precieved value for holding good 1 , 2 , 3)
        if self.consume_type == 3:
            self.Q_1 = 290 + random.randint(1, 10)
        if self.consume_type == 3:
            self.Q_2 = 230 + random.randint(1, 10)
        if self.consume_type == 3:
            self.Q_3 = 290 + u + random.randint(1, 10)

        # other characteristics wait to be added
        Agent.agentCount += 1

    def displayAgent(self):
        print("Agent id:", self.id, "consume:", self.consume_type, ",produce:", self.product_type, ",storage:",
              self.storage_type, "Q_1:", self.Q_1, "Q_2:", self.Q_2, "Q_3:", self.Q_3,"trading:",self.trading)


        
#Create agent one by one, with the basic setup: agents start with their own consumption good in hands
if __name__ == '__main__':   
    mylist = list()
    agent1 = Agent(1,1,2)
    mylist.append(agent1)
    agent2 = Agent(2,1,2)
    mylist.append(agent2)
    agent3 = Agent(3,1,2)
    mylist.append(agent3)
    agent4 = Agent(4,1,2)
    mylist.append(agent4)
    agent5 = Agent(5,1,2)
    mylist.append(agent5)
    agent6 = Agent(6,1,2)
    mylist.append(agent6)
    agent7 = Agent(7,1,2)
    mylist.append(agent7)
    agent8 = Agent(8,1,2)
    mylist.append(agent8)
    agent9 = Agent(9,2,3)
    mylist.append(agent9)
    agent10 = Agent(10,2,3)
    mylist.append(agent10)
    agent11 = Agent(11,2,3)
    mylist.append(agent11)
    agent12 = Agent(12,2,3)
    mylist.append(agent12)
    agent13 = Agent(13,2,3)
    mylist.append(agent13)
    agent14 = Agent(14,2,3)
    mylist.append(agent14)
    agent15 = Agent(15,2,3)
    mylist.append(agent15)
    agent16 = Agent(16,2,3)
    mylist.append(agent16)
    agent17 = Agent(17,3,1)
    mylist.append(agent17)
    agent18 = Agent(18,3,1)
    mylist.append(agent18)
    agent19 = Agent(19,3,1)
    mylist.append(agent19)
    agent20 = Agent(20,3,1)
    mylist.append(agent20)
    agent21 = Agent(21,3,1)
    mylist.append(agent21)
    agent22 = Agent(22,3,1)
    mylist.append(agent22)
    agent23 = Agent(23,3,1)
    mylist.append(agent23)
    agent24 = Agent(24,3,1)
    mylist.append(agent24)       

In [93]:
#matching process
def match(mylist):
    random.shuffle(mylist)
    matchlist = dict()
    index = list(range(0, len(mylist), 2))
    for i in index:
        matchlist[mylist[i]] = mylist[i+1]
    return matchlist
 

In [94]:
#TEST2 
for i in mylist:
    i.displayAgent()

Agent id: 1 consume: 1 ,produce: 2 ,storage: 2 Q_1: 167 Q_2: 71 Q_3: 112
Agent id: 2 consume: 1 ,produce: 2 ,storage: 2 Q_1: 173 Q_2: 69 Q_3: 108
Agent id: 3 consume: 1 ,produce: 2 ,storage: 2 Q_1: 166 Q_2: 71 Q_3: 109
Agent id: 4 consume: 1 ,produce: 2 ,storage: 2 Q_1: 171 Q_2: 67 Q_3: 104
Agent id: 5 consume: 1 ,produce: 2 ,storage: 2 Q_1: 169 Q_2: 67 Q_3: 106
Agent id: 6 consume: 1 ,produce: 2 ,storage: 2 Q_1: 167 Q_2: 73 Q_3: 105
Agent id: 7 consume: 1 ,produce: 2 ,storage: 2 Q_1: 167 Q_2: 67 Q_3: 106
Agent id: 8 consume: 1 ,produce: 2 ,storage: 2 Q_1: 173 Q_2: 66 Q_3: 111
Agent id: 9 consume: 2 ,produce: 3 ,storage: 3 Q_1: 264 Q_2: 351 Q_3: 250
Agent id: 10 consume: 2 ,produce: 3 ,storage: 3 Q_1: 259 Q_2: 346 Q_3: 252
Agent id: 11 consume: 2 ,produce: 3 ,storage: 3 Q_1: 263 Q_2: 347 Q_3: 247
Agent id: 12 consume: 2 ,produce: 3 ,storage: 3 Q_1: 261 Q_2: 350 Q_3: 251
Agent id: 13 consume: 2 ,produce: 3 ,storage: 3 Q_1: 261 Q_2: 352 Q_3: 250
Agent id: 14 consume: 2 ,produce: 3 ,stora

In [51]:
#matching process

random.shuffle(mylist)

matchlist = dict()
index = list(range(0, len(mylist), 2))
for i in index:
    matchlist[mylist[i]] = mylist[i+1]

for x, y in matchlist.items():
    print(x.id, y.id)

9 18
8 19
16 15
11 4
24 17
14 13
7 6
5 1
21 23
2 3
10 12
20 22


In [48]:
#trading decision
def trading_decision(Q_hold,C_hold,Q_trade,C_trade):
    if β*Q_hold - C_hold  < β*Q_trade - C_trade:
        de = 1
    else:
        de = 0
    return de

#Trading process
def trading_outcome(decision_1,decision_2):
    if decision_1 == 1 & decision_2 == 1:
        outcome = 1
    else:
        outcome = 0
    return outcome

In [27]:
#updating beliefs Q_ij
#just an example, need to be work for all agents

#γ = 1/t #decreasing gain
γ = 0.8 

if meeting_type == consumption_type & trading worked:
    V_12_prime = -c_2 + u + βQ_12
elif meeting_type == another_good_type & trading worked:
    V_13_prime = -c_3 + βQ_13
else:
    V_12_prime = -c_2 + βQ_12

Q_12 = Q_12 + γ(V_12_prime - Q_12)
Q_13 = Q_13 + γ(V_13_prime - Q_13)
Q_21 = Q_21 + γ(V_21_prime - Q_21)
Q_23 = Q_23 + γ(V_23_prime - Q_23)
Q_31 = Q_31 + γ(V_31_prime - Q_31)
Q_32 = Q_31 + γ(V_31_prime - Q_31)

SyntaxError: invalid syntax (2961504711.py, line 7)

In [75]:
#game stopping rule and restart a new round
def stopping_rule():
    stopping_draw = random.uniform(0,1)

    if stopping_draw > 0.1:
        #move to next round
        game_continue = 1
    else:
        #this game end
        game_continue = 0
    return game_continue
    
next_round = stopping_rule()
print(next_round)

#start a new round
if next_round == 0:
    #start a new game
    #keep everthing the same, but the inventory
    #the inventory resets to initial setting
    pass


1


In [86]:
#testing
class Agent:
    agentCount = 0
    def __init__(self,id,consume_type,product_type):
        self.id = id
        self.consume_type = consume_type
        self.product_type = product_type
        self.storage_type = product_type
        self.istrading = fasle;
        if self.consume_type == 1:
            self.Q_1 = 65 + random.randint(1,10)
        #other characteristics wait to be added 
        Agent.agentCount += 1
    def displayAgent(self):
        print("Agent id:",self.id,"consume:", self.consume_type, ",produce:", self.product_type, ",storage:",
              self.storage_type,"Q1:",self.Q_1)    
    
if __name__ == '__main__':   
    mylist = list()
    agent1 = Agent(1,1,2)

agent1.displayAgent()

Agent id: 1 consume: 1 ,produce: 2 ,storage: 2 Q1: 72


In [ ]:
#whats next:
#1. how two agents random match at each period
#1.1 after match if storage = same, then both do nothing
#1.2 if storage =! same, then each agent use their own parameters to decide trade or not

#2 Other agents properties 
#2.1 each agents has there own perceived value of other goods Q(at least the non consumption good)

#3 need to record agents behavior (maybe this is why agent-base is better?)
#3.1 when type 1 agent faced trading opportunity for good 3
#3.2 when type 2 agent faced trading opportunity for good 1
#3.3 when type 3 agent faced trading opportunity for good 2 

#4 how agents decide whether trade or not

#5
